In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import openai
from dotenv import load_dotenv, find_dotenv

In [3]:
def navigate_up(path, levels):
    """Navigate up `levels` directories from the given path."""
    for _ in range(levels):
        path = os.path.dirname(path)
    return path

In [4]:
# Get the current working directory
llamaindex_dir = os.getcwd()
# Get the parent directory
llamaindex_dir = os.path.dirname(llamaindex_dir)

sys.path.append(llamaindex_dir + "/utils")
sys.path.append(navigate_up(llamaindex_dir, 2) + "/law-sec-insights/backend")
# sys.path

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [5]:
from llamaindex_utils import *

In [6]:
from app.chat.engine import get_chat_engine

In [20]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Load data and build an index + Storing your index + Query your data

In [8]:
# from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage, get_response_synthesizer
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.postprocessor import SimilarityPostprocessor

# documents = SimpleDirectoryReader(llamaindex_dir + "/data").load_data()
# index = VectorStoreIndex.from_documents(documents)

In [9]:
# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader(llamaindex_dir + "/data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    # saving the embeddings to disk
    # By default, this will save the data to the directory storage, but you can change that by passing a persist_dir parameter.
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

```python
# Either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)
```

How to get lot of data when you have relevant results but potentially no data if you have nothing relevant
- we customize our retriever to use a different number for top_k
  - For a custom retriever, we use `RetrieverQueryEngine`.
- and add a post-processing step that requires that the retrieved nodes reach a minimum similarity score to be included
  - For the post-processing step, we use `SimilarityPostprocessor`
 

[Response Synthesizer](https://docs.llamaindex.ai/en/stable/module_guides/querying/response_synthesizers/): A Response Synthesizer is what generates a response from an LLM, using a user query and a given set of text chunks. The output of a response synthesizer is a Response object. When used in a query engine, the response synthesizer is used after nodes are retrieved from a retriever, and after any node-postprocessors are ran.

In [10]:
# build index
# index = VectorStoreIndex.from_documents(documents)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

# query
response = query_engine.query("What did the author do growing up?")
print(response)

The author started writing essays again and even worked on Lisp in March 2015.


In [11]:
# help(RetrieverQueryEngine)

# `get_chat_engine`

- local implementation

There are a huge variety of retrievers that you can learn about in our [module guide on retrievers](https://docs.llamaindex.ai/en/stable/module_guides/querying/retriever/).

In [69]:
from app.models.db import MessageRoleEnum, MessageStatusEnum, MessageSubProcess, MessageSubProcessStatusEnum
from app.schema import Message, Conversation, Document, DocumentMetadataKeysEnum, SecDocumentTypeEnum
from datetime import datetime
from uuid import UUID


mock_message = Message(
    conversation_id=UUID("01234567-89ab-cdef-0123-456789abcdef"),
    content="Hello, how can I help you?",
    role=MessageRoleEnum.assistant,
    status=MessageStatusEnum.SUCCESS,
    sub_processes=[
        MessageSubProcess(
            message_id=UUID("01234567-89ab-cdef-0123-456789abcdef"),
            source="chunking",
            status=MessageSubProcessStatusEnum.FINISHED,
            metadata_map=None
        ),
        MessageSubProcess(
            message_id=UUID("abcdef01-2345-6789-abcd-ef0123456789"),
            source="node_parsing",
            status=MessageSubProcessStatusEnum.FINISHED,
            metadata_map=None
        )
    ]
)

mock_document = Document(
    id=UUID("123e4567-e89b-12d3-a456-426614174000"),
    created_at=datetime.now(),
    updated_at=datetime.now(),
    url="https://example.com/document.pdf",
    metadata_map={
        DocumentMetadataKeysEnum.SEC_DOCUMENT: {
            "title": "Annual Report 2023",
            "author": "Company XYZ",
            "pages": 25,
            "company_name": "Apple",
            "company_ticker": "AAPL",
            "doc_type": SecDocumentTypeEnum.TEN_K,
            "year": 2023
            
        }
    }
)

index.set_index_id(str(mock_document.id))
doc_id_to_index = {"123e4567-e89b-12d3-a456-426614174000": index}

In [70]:
mock_document.id

UUID('123e4567-e89b-12d3-a456-426614174000')

In [71]:
from app.schema import Conversation as ConversationSchema

conversation = ConversationSchema(messages=[mock_message], documents=[mock_document])

In [72]:
from app.chat.utils import build_title_for_document

if conversation.documents:
    doc_titles = "\n".join(
        "- " + build_title_for_document(doc) for doc in conversation.documents
    )
else:
    doc_titles = "No documents selected."

doc_titles

'- Apple (AAPL) 10-K (2023)'

In [73]:
from app.chat.notebook_engine import get_chat_engine

[Llama Debug Handler](https://docs.llamaindex.ai/en/stable/examples/callbacks/LlamaDebugHandler/)

In [74]:
from llama_index.callbacks import LlamaDebugHandler

llama_debug = LlamaDebugHandler(print_trace_on_end=True)

agent = await get_chat_engine(conversation=conversation, doc_id_to_index=doc_id_to_index, callback_handler=llama_debug)

# Debugging

In [75]:
# response = agent.chat("What did he work on outside of school?")
# response = agent.chat("What did he work on outside of school?")
response = agent.chat("What did the author do growing up?")
# TODO:
# FIX FOR THIS MOCK

STARTING TURN 1
---------------

**********
Trace: chat
    |_CBEventType.AGENT_STEP ->  1.226806 seconds
      |_CBEventType.LLM ->  1.226142 seconds
**********


In [76]:
response

AgentChatResponse(response="I'm sorry, but I don't have any information about the author's personal life or upbringing. My capabilities are limited to providing general information and answering questions based on the data I have been trained on. Is there anything else I can assist you with?", sources=[], source_nodes=[])

In [61]:
# iterate through sub_question items captured in SUB_QUESTION event
from llama_index.callbacks import CBEventType, EventPayload

def print_subquestion_debug(llama_debug):
    for i, (start_event, end_event) in enumerate(
        llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)
    ):
        qa_pair = end_event.payload[EventPayload.SUB_QUESTION]
        print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
        print("Answer: " + qa_pair.answer.strip())
        print("====================================")

print_subquestion_debug(llama_debug)

Sub Question 0: What is the employment history of the person?
Answer: Empty Response
Sub Question 1: What is the author's name?
Answer: Empty Response
Sub Question 2: What is the author's age?
Answer: Empty Response
Sub Question 3: What is the author's educational background?
Answer: Empty Response
Sub Question 4: What is the author's career history?
Answer: Empty Response


In [56]:
response

AgentChatResponse(response="I'm sorry, but I don't have any information about what he worked on outside of school. Is there anything else I can help you with?", sources=[ToolOutput(content='Empty Response', tool_name='qualitative_question_engine', raw_input={'input': 'What did he work on outside of school?'}, raw_output=Response(response='Empty Response', source_nodes=[NodeWithScore(node=TextNode(id_='53cc4700-713a-4742-919c-868b851b120c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='7d5238a1bed65efaf5322516742e8a633b871cb5497a61c2d7ffff96f2b2374f', text='Sub question: What is the employment history of the person?\nResponse: Empty Response', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'53cc4700-713a-4742-919c-868b851b120c': {}}))], source_nodes=[NodeWithScore(node=TextNode(id_='53cc4700-713

In [26]:
from llama_index.callbacks import CBEventType

print(llama_debug.get_event_time_info(CBEventType.LLM))

EventStats(total_secs=1.101996, average_secs=1.101996, total_count=1)


In [32]:
# Print info on llm inputs/outputs - returns start/end events for each LLM call
event_pairs = llama_debug.get_llm_inputs_outputs()
event_pairs[0][0].payload

{<EventPayload.MESSAGES: 'messages'>: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content="You are an expert financial analyst that always answers questions with the most relevant information using the tools at your disposal.\nThese tools have information regarding companies that the user has expressed interest in.\nHere are some guidelines that you must follow:\n* For financial questions, you must use the tools to find the answer and then write a response.\n* Even if it seems like your tools won't be able to answer the question, you must still use them to find the most relevant information and insights. Not using them will appear as if you are not doing your job.\n* You may assume that the users financial questions are related to the documents they've selected.\n* For any user message that isn't related to financial analysis, respectfully decline to respond and suggest that the user ask a relevant question.\n* If your tools are unable to find an answer, you should say that you h

In [37]:
event_pairs[0][0].payload.keys()

dict_keys([<EventPayload.MESSAGES: 'messages'>, <EventPayload.ADDITIONAL_KWARGS: 'additional_kwargs'>, <EventPayload.SERIALIZED: 'serialized'>])

In [29]:
print(event_pairs[0][1].payload.keys())

dict_keys([<EventPayload.MESSAGES: 'messages'>, <EventPayload.RESPONSE: 'response'>])


In [30]:
print(event_pairs[0][1].payload["response"])

assistant: I'm sorry, but I'm not sure who you are referring to. Could you please provide more context or clarify your question?


In [33]:
# content for role=<MessageRole.SYSTEM: 'system'>
print("You are an expert financial analyst that always answers questions with the most relevant information using the tools at your disposal.\nThese tools have information regarding companies that the user has expressed interest in.\nHere are some guidelines that you must follow:\n* For financial questions, you must use the tools to find the answer and then write a response.\n* Even if it seems like your tools won't be able to answer the question, you must still use them to find the most relevant information and insights. Not using them will appear as if you are not doing your job.\n* You may assume that the users financial questions are related to the documents they've selected.\n* For any user message that isn't related to financial analysis, respectfully decline to respond and suggest that the user ask a relevant question.\n* If your tools are unable to find an answer, you should say that you haven't found an answer but still relay any useful information the tools found.\n\nThe tools at your disposal have access to the following SEC documents that the user has selected to discuss with you:\n- Apple (AAPL) 10-K (2023)\n\nThe current date is: 2024-05-01")

You are an expert financial analyst that always answers questions with the most relevant information using the tools at your disposal.
These tools have information regarding companies that the user has expressed interest in.
Here are some guidelines that you must follow:
* For financial questions, you must use the tools to find the answer and then write a response.
* Even if it seems like your tools won't be able to answer the question, you must still use them to find the most relevant information and insights. Not using them will appear as if you are not doing your job.
* You may assume that the users financial questions are related to the documents they've selected.
* For any user message that isn't related to financial analysis, respectfully decline to respond and suggest that the user ask a relevant question.
* If your tools are unable to find an answer, you should say that you haven't found an answer but still relay any useful information the tools found.

The tools at your dispos

In [ ]:
# ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Hello, how can I help you?', additional_kwargs={}),
# ChatMessage(role=<MessageRole.USER: 'user'>, content='What did he work on outside of school?', additional_kwargs={})],

In [40]:
event_pairs[0][0]

CBEvent(event_type=<CBEventType.LLM: 'llm'>, payload={<EventPayload.MESSAGES: 'messages'>: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content="You are an expert financial analyst that always answers questions with the most relevant information using the tools at your disposal.\nThese tools have information regarding companies that the user has expressed interest in.\nHere are some guidelines that you must follow:\n* For financial questions, you must use the tools to find the answer and then write a response.\n* Even if it seems like your tools won't be able to answer the question, you must still use them to find the most relevant information and insights. Not using them will appear as if you are not doing your job.\n* You may assume that the users financial questions are related to the documents they've selected.\n* For any user message that isn't related to financial analysis, respectfully decline to respond and suggest that the user ask a relevant question.\n* If your tools ar

In [41]:
event_pairs[0][1]

CBEvent(event_type=<CBEventType.LLM: 'llm'>, payload={<EventPayload.MESSAGES: 'messages'>: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content="You are an expert financial analyst that always answers questions with the most relevant information using the tools at your disposal.\nThese tools have information regarding companies that the user has expressed interest in.\nHere are some guidelines that you must follow:\n* For financial questions, you must use the tools to find the answer and then write a response.\n* Even if it seems like your tools won't be able to answer the question, you must still use them to find the most relevant information and insights. Not using them will appear as if you are not doing your job.\n* You may assume that the users financial questions are related to the documents they've selected.\n* For any user message that isn't related to financial analysis, respectfully decline to respond and suggest that the user ask a relevant question.\n* If your tools ar

In [52]:
from llama_index.callbacks import CBEventType, EventPayload
import pprint

for i, (start_event, end_event) in enumerate(
    llama_debug.get_event_pairs(CBEventType.LLM)
):
    # qa_pair = end_event.payload[CBEventType.LLM]
    print("\n\n---------------- start_event ----------------")
    pprint.pprint(start_event.payload[EventPayload.MESSAGES])
    print("\n\n---------------- end_event ----------------")
    # print(end_event)
    # print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
    # print("Answer: " + qa_pair.answer.strip())
    # print("====================================")



---------------- start_event ----------------
[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content="You are an expert financial analyst that always answers questions with the most relevant information using the tools at your disposal.\nThese tools have information regarding companies that the user has expressed interest in.\nHere are some guidelines that you must follow:\n* For financial questions, you must use the tools to find the answer and then write a response.\n* Even if it seems like your tools won't be able to answer the question, you must still use them to find the most relevant information and insights. Not using them will appear as if you are not doing your job.\n* You may assume that the users financial questions are related to the documents they've selected.\n* For any user message that isn't related to financial analysis, respectfully decline to respond and suggest that the user ask a relevant question.\n* If your tools are unable to find an answer, you should say 